## pyMAISE Initialization

First we initialize pyMAISE with the following 4 parameters:

- `verbosity`: 0 $\rightarrow$ pyMAISE prints no outputs,
- `random_state`: None $\rightarrow$ No random seed is set,
- `test_size`: 0.3 $\rightarrow$ 30% of the data is used for testing,
- `num_configs_saved`: 5 $\rightarrow$ The top 5 hyper-parameter configurations are saved for each model.

With pyMAISE initialized we can load the preprocessor for this data set using `load_fp()`.

In [2]:
import pyMAISE as mai
import pytest
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.stats import uniform, randint
from sklearn.model_selection import ShuffleSplit

# Regression test parameters
# Data set parameters
num_observations = 150
num_features = 4
num_outputs = 1

expected_models = {
    "dtree": 1.0,
    "rforest": 1.0,
    "knn": 1.0,
    
}

# Expected Model Settings
matplotlib_settings = {
    "font.size": 14,
    "legend.fontsize": 12,
}
plus_minus= 0.025

settings = { 
    "verbosity": 0,
    "random_state": 42,
    "test_size": 0.3,
    "num_configs_saved": 1,
    "regression": False,
    "classification": True,
}


global_settings = mai.settings.init(settings_changes=settings)





preprocessor = mai.load_iris()
data = preprocessor.data_split()

model_settings = { "models": ["logistic", "dtree", "rforest", "knn"],
                 }


tuning = mai.Tuning(data=data, model_settings=model_settings)

grid_search_spaces = {
    "logistic":{
        "penalty": ['l1', 'l2', 'elasticnet'],
        "dual": [True, False],
        "tol": [0.001],
    },
    "dtree":{
        "max_depth": [None, 5, 10, 25, 50],
        "max_features": [None, "sqrt", "log2"],
        "min_samples_leaf": [1, 2, 4, 6, 8, 10],
        "min_samples_split": [2, 4, 6, 8, 10],
    },
    "rforest": {
        "n_estimators": [50, 100, 150],
        "criterion": ["gini", "entropy", "log_loss"],
        "min_samples_split": [2, 4, 6],
        "max_features": ["sqrt", "log2"],
                      
    },
    "knn": {
        "n_neighbors": [1, 2, 4, 6, 8, 10, 14, 17, 20],
        "weights": ["uniform", "distance"],
        "leaf_size": [1, 5, 10, 15, 20, 30],
    },
}
grid_search_configs = tuning.grid_search(param_spaces=grid_search_spaces, models=grid_search_spaces.keys(), cv=ShuffleSplit(n_splits=1, test_size=0.15, random_state=global_settings.random_state),)

postprocessor = mai.PostProcessor(
    data=data,
    models_list=[grid_search_configs],
)

for key, value in expected_models.items():
    assert postprocessor.metrics(model_type=key)["Test Accuracy"].to_numpy()[0] == pytest.approx(value, plus_minus / value)

2023-09-10 21:57:32.278251: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 21:57:35.692809: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 21:57:35.773355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-10 21:57:48.740098: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-10 21:57:51.442519: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-10 21:57:51.444322: W tensorflow/core/common_r

In [3]:
postprocessor.metrics()

,Model Types,Parameter Configurations,Train Accuracy,Train Recall,Train Precision,Train F1,Test Accuracy,Test Recall,Test Precision,Test F1
0,logistic,"{'dual': False, 'penalty': 'l2', 'tol': 0.001}",0.961905,0.961905,0.961905,0.961905,1.0,1.0,1.0,1.0
1,dtree,"{'max_depth': None, 'max_features': None, 'min...",1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
2,rforest,"{'criterion': 'gini', 'max_features': 'sqrt', ...",1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0
3,knn,"{'leaf_size': 10, 'n_neighbors': 20, 'weights'...",1.000000,1.000000,1.000000,1.000000,1.0,1.0,1.0,1.0


# New NN Object using MIT Reactor Data for example

In [23]:
import pyMAISE as mai
from pyMAISE.methods import *
import pytest
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from scipy.stats import uniform, randint
from sklearn.model_selection import ShuffleSplit
import keras_tuner
import tensorflow as tf
import kerastuner as kt
from kerastuner import HyperModel

import flatdict

from sklearn.metrics import r2_score

global_settings = mai.settings.init({"regression": True})

preprocessor = mai.load_MITR()

xtrain, xtest, ytrain, ytest = preprocessor.data_split()


print("--------- NEW TEST ---------------")
optimizer_hyperparameters = {
    # Optimizer
    "optimizer": "adam",
    "learning_rate": 0.0099,
    "epochs": 50,
    "batch_size": 27,
    "loss": "mean_absolute_error",
    "metrics": ["mean_absolute_error"],
    "warm_start": True,
    "jit_compile": False,
    
}

structural_hyperparameters = {
    "structural_hyperparameters":{
        "dense_input": {
            "num_nodes": 12, 
            "input_dim": preprocessor.inputs.shape[1],
            "activation": 'relu',
            },
        "dense": {
            "num_nodes": 15,
            "shape": None,
            "activation": 'relu',
        },
        "dense_output": {
            "num_nodes": 118,
            "shape": preprocessor.outputs.shape[1],
            "activation": 'relu',
        },
    },
    
}
# An example that we ould do to flatten the dictionary for the potential issue you said about milti dim dictionary
d =  flatdict.FlatDict(structural_hyperparameters, delimiter='.')
print(d)

#-----------------------------------------------------------------------------------
# Using the Baysian Search I saw

print("--------- NEW TEST ---------------")

hp = keras_tuner.HyperParameters()


optimizer_hyperparameters = {
    # Optimizer
    "optimizer": "adam",
    "learning_rate": 1e-4, # hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, default=1e-3):w
    "epochs": 50,
    "batch_size": 27,
    "loss": "mean_absolute_error",
    "metrics": ["mean_absolute_error"],
    "warm_start": True,
    "jit_compile": False,
    
}

structural_hyperparameters = {
    "structural_hyperparameters":{
        "dense_input": {
            "num_nodes": 100, # hp.Int('units1', min_value=50, max_value=350, step=50), 
            "input_dim": preprocessor.inputs.shape[1],
            "activation": 'relu',
            },
        "dense": {
            "num_nodes": [50, 100, 5], # hp.Int('units2', min_value=50, max_value=100, step=25),
            "input_dim": None,
            "activation": 'relu',
        },
        "dense_output": {
            "num_nodes":  preprocessor.outputs.shape[1], # hp.Int('units3', min_value=50, max_value=100, step=25),
            "input_dim": None,
            "activation": 'relu',
        },
    },
    
}



build_model = nnHyperModel(structural_hyperparameters, optimizer_hyperparameters)

tuner = kt.BayesianOptimization(
        build_model,
        objective="mean_absolute_error",
        max_trials=10,
        seed=42)
print("[INFO] performing hyperparameter search...")
tuner.search(
    x=xtrain,
    y=ytrain,
    validation_data=(xtest, ytest),
    batch_size=50,
    epochs=10,
)
# grab the best hyperparameters
bestHP = tuner.get_best_hyperparameters(num_trials=1)[0]
print("[INFO] optimal number of units in dense layer: {}".format(
	bestHP.get("Units")))

# build the best model and train it
print("[INFO] training the best model...")
model = tuner.hypermodel.build(bestHP)
H = model.fit(x=xtrain, y=ytrain,
validation_data=(xtest, ytest), batch_size=32,
epochs=50, verbose=1)
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(x=xtest, batch_size=32)
print(r2_score(ytest, predictions))





--------- NEW TEST ---------------
{'structural_hyperparameters.dense_input.num_nodes': 12, 'structural_hyperparameters.dense_input.input_dim': 6, 'structural_hyperparameters.dense_input.activation': 'relu', 'structural_hyperparameters.dense.num_nodes': 15, 'structural_hyperparameters.dense.shape': None, 'structural_hyperparameters.dense.activation': 'relu', 'structural_hyperparameters.dense_output.num_nodes': 118, 'structural_hyperparameters.dense_output.shape': 22, 'structural_hyperparameters.dense_output.activation': 'relu'}
--------- NEW TEST ---------------
[INFO] performing hyperparameter search...
[INFO] optimal number of units in dense layer: 85
[INFO] training the best model...
Epoch 1/50
22/22 [==============================] - 16s 533ms/step - loss: 19886.1621 - mean_absolute_error: 19886.1621 - val_loss: 19885.4629 - val_mean_absolute_error: 19885.4629
Epoch 2/50
22/22 [==============================] - 8s 377ms/step - loss: 19885.3066 - mean_absolute_error: 19885.3066 - va

22/22 [==============================] - 21s 993ms/step - loss: 18940.7090 - mean_absolute_error: 18940.7090 - val_loss: 18908.3750 - val_mean_absolute_error: 18908.3750
Epoch 43/50
22/22 [==============================] - 25s 1s/step - loss: 18872.7793 - mean_absolute_error: 18872.7793 - val_loss: 18839.0234 - val_mean_absolute_error: 18839.0234
Epoch 44/50
22/22 [==============================] - 31s 1s/step - loss: 18801.7598 - mean_absolute_error: 18801.7598 - val_loss: 18766.6270 - val_mean_absolute_error: 18766.6270
Epoch 45/50
22/22 [==============================] - 24s 1s/step - loss: 18727.6562 - mean_absolute_error: 18727.6562 - val_loss: 18691.1582 - val_mean_absolute_error: 18691.1582
Epoch 46/50
22/22 [==============================] - 28s 1s/step - loss: 18650.4707 - mean_absolute_error: 18650.4707 - val_loss: 18612.5391 - val_mean_absolute_error: 18612.5391
Epoch 47/50
22/22 [==============================] - 26s 1s/step - loss: 18570.1074 - mean_absolute_error: 18570.1